Harika—bu RBM örneğini hem **adım adım açıklayacağım**, hem de **daha sağlam (ufak düzeltmelerle)** çalışan bir örnek kodu paylaşacağım. Kod MovieLens **ml-100k** (u1.base / u1.test) üzerinde **Bernoulli–Bernoulli RBM** ile **beğeni (>=3)** / **beğenmedi (1–2)** ikilileştirilmiş (binary) tahmin yapıyor.

---

# 1) Verdiğin kod ne yapıyor? (Adım adım)

### (a) Veriyi okuma ve kullanıcı–film matrisine dönüştürme

```python
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
test_set     = pd.read_csv('ml-100k/u1.test', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set     = np.array(test_set, dtype='int')

nb_users  = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
```

* `u1.base` ve `u1.test` “(user, movie, rating, timestamp)” satırlarıdır.
* `nb_users`, `nb_movies` ile kullanıcı/film sayısını alıyor.

```python
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies  = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)  # dolu olmayanlar = 0
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)  # (nb_users, nb_movies)
test_set     = convert(test_set)
training_set = torch.FloatTensor(training_set)
test_set     = torch.FloatTensor(test_set)
```

* **Yoğun** (dense) bir **kullanıcı × film** matrisi yapıyor: her satır bir kullanıcı, sütunlar filmler, değerler rating (0 = hiç oy yok).

### (b) Binarizasyon ve “eksik”lerin işaretlenmesi

```python
training_set[training_set == 0]  = -1  # "hiç oy yok" -> -1 (mask işareti)
training_set[training_set == 1]  = 0   # 1-2 -> 0 (beğenmedi)
training_set[training_set == 2]  = 0
training_set[training_set >= 3]  = 1   # 3-4-5 -> 1 (beğendi)

test_set[test_set == 0]  = -1
test_set[test_set == 1]  = 0
test_set[test_set == 2]  = 0
test_set[test_set >= 3]  = 1
```

* RBM görünür birimleri **Bernoulli** kabul ettiği için rating’leri **0/1**’e çeviriyor; **eksikler** `-1` ile işaretleniyor.

### (c) RBM sınıfı (Bernoulli–Bernoulli)

```python
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)  # ağırlıklar (hidden x visible)
        self.a = torch.randn(1, nh)   # gizli bias
        self.b = torch.randn(1, nv)   # görünür bias
```

* `nv` = film sayısı (visible units), `nh` = gizli ünite sayısı.

```python
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
```

* **Koşullu** olasılıklar: $p(h|v)$ ve $p(v|h)$ sigmoid; ve Bernoulli örnekleme ile ikili değer üretiliyor.

```python
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0).t() - torch.mm(vk.t(), phk).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
```

* **Contrastive Divergence** güncellemesi:
  **pozitif faz** $v_0, p(h|v_0)$ – **negatif faz** $v_k, p(h|v_k)$.
  (Not: burada **öğrenme oranı yok**, **batch’e bölünmüyor**; pratikte LR ve ‘/batch\_size’ eklemek iyidir.)

### (d) Eğitim döngüsü (CD-10, mini-batch=100 kullanıcı)

```python
nv = len(training_set[0]); nh = 100; batch_size = 100
rbm = RBM(nv, nh)
nb_epoch = 10

for epoch in range(1, nb_epoch + 1):
    train_loss = 0; s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]  # negatif zincire girecek kopya
        v0 = training_set[id_user:id_user+batch_size]  # orijinal batch
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):   # CD-10
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]  # eksik olanları (-1) olduğu gibi bırak
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch:', epoch, 'loss:', train_loss/s)
```

* **CD-10** zinciri: $v_0 \to h_0 \to v_1 \to h_1 \dots \to v_{10}$.
* `vk[v0<0]=v0[v0<0]` ile **eksik** değerleri zincir boyunca **sabit tutuyor** (rekonstrüksiyon ve kayıp bu noktalarda hesaplanmıyor).
* Kayıp: **bilinen** girdilerde $|v_0 - v_k|$ (MAE).

### (e) Test

```python
test_loss = 0; s = 0.
for id_user in range(nb_users):
    v  = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)  # rekonstrüksiyon
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss:', test_loss/s)
```

* Her kullanıcı için test setindeki **bilinen** filmlerde MAE bakıyor.

---

# 2) Önemli notlar / iyileştirmeler

1. **Öğrenme oranı (lr) ve batch bölme**
   Kod, ağırlıkları **ölçeksiz** güncelliyor. Pratikte:

   $$
   W \leftarrow W + \eta \frac{(v_0^\top p(h|v_0) - v_k^\top p(h|v_k))^\top}{\text{batch\_size}}
   $$

   ve bias’ları da `η/batch_size` ile güncellemek daha stabil.

2. **Eksikleri -1 ile beslemek**
   `sample_h(v0)` çağrısında `v0` içinde **-1**’ler var; bu, Bernoulli RBM varsayımıyla **tutarlı değil**.
   Daha doğru yaklaşım: **mask** tut, **pozitif faz** girişini `v0_missing→0` yaparak ver (veya inputu 0/1’e çevirip eksikleri 0 say), kaybı yalnız **bilinen**lerde hesapla.

3. **Başlatma ve düzenleme (regularization)**

   * Ağırlıkları **Xavier/He** gibi küçük ölçekle başlat.
   * **Momentum / weight decay** eklemek işe yarar.

4. **Mini-batch & shuffle**
   Kullanıcıları her epoch **karıştır** (shuffle). DataLoader ile daha temiz.

5. **Metrikler**
   İkili yapıdaysan testte **AUC / Precision\@K / Recall\@K** gibi metrikler daha anlamlı olabilir.

---

# 3) Düzeltilmiş ve açıklamalı örnek kod

Aşağıdaki sürüm, aynı fikri korur ama:

* **LR** ve **/batch\_size** ekler,
* **mask** kullanır: eksikleri modele **0** verip, **pozitif faz** ve kaybı yalnız **bilinen**lerde hesaplar,
* **shuffle** ile DataLoader kullanır,
* `cd_k` parametreleştirir (CD-1 çoğu zaman yeterli).

> `pip install torch pandas` (MovieLens dosyaları aynı konumda olmalı)

```python
# rbm_movielens_clean.py
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# 1) Veri: u1.base / u1.test -> kullanıcı×film matrisi
def load_ml100k(base_path="ml-100k"):
    train = pd.read_csv(f"{base_path}/u1.base", delimiter="\t", header=None, names=["u","i","r","t"]).values
    test  = pd.read_csv(f"{base_path}/u1.test", delimiter="\t", header=None, names=["u","i","r","t"]).values
    nb_users  = int(max(train[:,0].max(), test[:,0].max()))
    nb_items  = int(max(train[:,1].max(), test[:,1].max()))

    def convert(data):
        X = np.zeros((nb_users, nb_items), dtype=np.float32)
        for u in range(1, nb_users+1):
            movies  = data[:,1][data[:,0]==u]
            ratings = data[:,2][data[:,0]==u]
            X[u-1, movies-1] = ratings
        return X

    Xtr = convert(train)  # rating: 0 (none) or 1..5
    Xte = convert(test)
    return Xtr, Xte

Xtr, Xte = load_ml100k()

# 2) Binarize: 1-2 -> 0, 3-4-5 -> 1; missing(0) -> mask
def binarize(X):
    X_bin = X.copy()
    mask  = (X_bin > 0)  # known ratings
    X_bin[X_bin <= 2] = 0
    X_bin[X_bin >= 3] = 1
    # Eksikleri 0 olarak bırakıyoruz; mask ile ayrıştıracağız
    return X_bin.astype(np.float32), mask.astype(np.float32)

Vtr, Mtr = binarize(Xtr)
Vte, Mte = binarize(Xte)

Vtr_t = torch.from_numpy(Vtr)  # (U, I)
Mtr_t = torch.from_numpy(Mtr)  # (U, I)
Vte_t = torch.from_numpy(Vte)
Mte_t = torch.from_numpy(Mte)

# 3) RBM sınıfı (Bernoulli-Bernoulli)
class RBM:
    def __init__(self, nv, nh, lr=0.01, cd_k=1, device="cpu"):
        self.nv, self.nh = nv, nh
        self.lr   = lr
        self.cd_k = cd_k
        self.W = torch.randn(nh, nv, device=device) * 0.01
        self.a = torch.zeros(1, nh, device=device)   # hidden bias
        self.b = torch.zeros(1, nv, device=device)   # visible bias
        self.device = device

    def sample_h(self, v):
        # v: (B, nv)
        wx = v @ self.W.t() + self.a.expand_as(v @ self.W.t())
        p_h = torch.sigmoid(wx)
        h   = torch.bernoulli(p_h)
        return p_h, h

    def sample_v(self, h):
        wy = h @ self.W + self.b.expand_as(h @ self.W)
        p_v = torch.sigmoid(wy)
        v   = torch.bernoulli(p_v)
        return p_v, v

    @torch.no_grad()
    def train_step(self, v0, mask):
        """
        v0: (B, nv)  -> 0/1, missingler 0 (mask=0)
        mask: (B, nv) -> 1: known, 0: missing
        CD-k: known pozisyonları 'clamp' etmek için mask kullanabiliriz.
        """
        B = v0.size(0)

        # Pozitif faz
        ph0, h0 = self.sample_h(v0)

        # Negatif faz (CD-k)
        vk = v0.clone()
        for _ in range(self.cd_k):
            ph, h = self.sample_h(vk)
            pv, v = self.sample_v(h)
            # İsteğe bağlı: known pozisyonları orijinalde sabit tut
            vk = vk * (1 - mask) + v * mask  # clamp knowns to model sample? (alternatif: vk[mask==1]=v0[mask==1])

        phk, _ = self.sample_h(vk)

        # Ağırlık güncelleme (lr ve batch bölme ile)
        self.W += self.lr * ((v0.t() @ ph0) - (vk.t() @ phk)).t() / B
        self.b += self.lr * torch.sum((v0 - vk), dim=0, keepdim=True) / B
        self.a += self.lr * torch.sum((ph0 - phk), dim=0, keepdim=True) / B

        # Kayıp: yalnız knownlarda MAE
        loss = torch.mean(torch.abs((v0 - vk)[mask.bool()]))
        return loss.item()

# 4) Eğitim
device = "cuda" if torch.cuda.is_available() else "cpu"
nv = Vtr_t.size(1)
rbm = RBM(nv=nv, nh=100, lr=0.05, cd_k=1, device=device)

dataset = TensorDataset(Vtr_t.to(device), Mtr_t.to(device))
loader  = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)

for epoch in range(10):
    losses = []
    for v0, mask in loader:
        loss = rbm.train_step(v0, mask)
        losses.append(loss)
    print(f"Epoch {epoch+1:02d} | train MAE(on known): {np.mean(losses):.4f}")

# 5) Test: rekonstrüksiyon hatası (sadece test setindeki knownlarda)
@torch.no_grad()
def eval_mae(rbm, V, M):
    V = V.to(rbm.device); M = M.to(rbm.device)
    ph, h = rbm.sample_h(V)
    pv, v = rbm.sample_v(h)
    mae = torch.mean(torch.abs((V - v)[M.bool()])).item()
    return mae

test_mae = eval_mae(rbm, Vte_t, Mte_t)
print(f"Test MAE (known entries): {test_mae:.4f}")
```

**Notlar (kod):**

* **Mask** ile eksikleri net biçimde ayırdık; **girişleri 0** olarak verip kaybı **yalnız bilinenlerde** ölçüyoruz.
* `train_step`’te **öğrenme oranı** ve **/batch\_size** var → daha kararlı.
* `cd_k=1` çoğu zaman yeter; istersen 5–10 deneyebilirsin (daha yavaş).
* İstersen `vk = v0 * mask + v * (1 - mask)` şeklinde **knownları tamamen clamp** edip sadece eksikleri üretmek de bir tercih (demo amaçlı yukarıdaki harman yöntemi bırakıldı).

---

## 4) Kapanış – özet

* RBM, **beğeni/ beğenmeme** gibi **ikili** görünür değişkenlerle iyi çalışır → MovieLens’i bu yüzden **binary** yaptık.
* Eğitimde asıl fikir: **pozitif faz** (veriyle hizalan) – **negatif faz** (modelin kendi örnekleriyle uzaklaş).
* Pratikte **lr/batch**, **mask**, **shuffle**, **cd\_k**, **başlatma** ve **regularization** modeli ciddi biçimde stabilize eder.

İstersen bir sonraki adımda **Gaussian–Bernoulli RBM** (sürekli görünürler için) veya **öneri metrikleri** (Precision\@K/Recall\@K, AUC) ekleyebiliriz.
